In [1]:
# importing prompts 
from scai.modules.task.prompts import TASK_PROMPTS
from scai.modules.assistant.prompts import ASSISTANT_PROMPTS 
from scai.modules.user.prompts import USER_PROMPTS 
from scai.modules.meta_prompt.prompts import META_PROMPTS 

# import episode
from scai.modules.episode.episode import Episode

# import llms
from scai.modules.custom_chat_models.crfm import crfmChatLLM # model


In [2]:
# Define hyperparameters
params = {
    "n_user": 2,
    "n_assistant": 2,
    "system_k": 5,
    "chat_k": 5,
    "user_k": 5,
    "assistant_k": 5,
    "assistant_system_k": 1,
}

# Define prompts
prompts = {
    "task": TASK_PROMPTS["task_prompt_1"],
    "user": [USER_PROMPTS["user_prompt_1"], USER_PROMPTS["user_prompt_2"]],
    "assistant": [ASSISTANT_PROMPTS["assistant_prompt_1"], ASSISTANT_PROMPTS["assistant_prompt_1"]],
    "meta": META_PROMPTS["meta_prompt_1"],
}

# Define system message
system_message_1 = "You are a helpful AI assistant."

# Create an episode
episode = Episode.create(
    id="episode_1",
    name="episode_1",
    n_user=params["n_user"],
    user_llm='userllm',
    n_assistant=params["n_assistant"],
    assistant_llm='assistantllm',
    meta_llm='metallm',
    system_k=params["system_k"],
    chat_k=params["chat_k"],
    user_k=params["user_k"],
    assistant_k=params["assistant_k"],
    assistant_system_k=params["assistant_system_k"],
)

# Save system message into the buffer
# episode.buffer.save_context(system={"content": system_message_1}, system_message_id="system_message_1")


TypeError: Episode() takes no arguments

In [5]:

# # initialise buffer 
# buffer = CustomConversationBufferWindowMemory(system_k=5, 
#                                               chat_k=5, 
#                                               user_k=5, 
#                                               assistant_k=5,
#                                               assistant_system_k=1,
#                                               ) # how many turns are stored will be used

# system_message_1 = "You are a helpful AI assistant :)."
# buffer.save_context(system={"content": system_message_1}, system_message_id="system_message_1")
# print(buffer.system_memory.messages)

In [6]:
# # instantiate conversations—one assistant for each user
# assistant_model_1 = AssistantModel(llm='chat_llm', 
#                                    conversation_id="conversation_1")
# assistant_response_1 = assistant_model_1.run(assistant_prompt=assistant_prompt, 
#                                              task_prompt=task_prompt,
#                                               buffer=buffer)



# assistant_model_2 = AssistantModel(llm='chat_llm', 
#                                    conversation_id="conversation_2")
# assistant_response_2 = assistant_model_2.run(assistant_prompt=assistant_prompt, 
#                                              task_prompt=task_prompt,
#                                              buffer=buffer)
# print("assistant response 2")
# print(assistant_response_2)

# print("assistant response 1")
# print(assistant_response_1)
# print()

In [7]:
# # initial assistant response
# buffer.save_context(assistant={"content": "AI should maintain impartiality, respect human rights and dignity, consider cultural nuances, and abide by local laws while handling sensitive topics like LGBTQ and women's rights."},
#                     assistant_message_id="conversation_1_assistant_message_1")

# buffer.save_context(assistant={"content": "AI should respect human rights universally while acknowledging cultural contexts. Responses shouldn't change values but can adapt communication style to ensure comprehension and respect."},
#                     assistant_message_id="conversation_2_assistant_message_1")

In [8]:
# user_model_1 = UserModel(llm='chat_llm', conversation_id="conversation_1")
# user_response_1 = user_model_1.run(user_prompt=user_prompt_1, 
#                                  task_prompt=task_prompt, 
#                                  buffer=buffer)

# print(user_response_1)

In [9]:
# user_model_2 = UserModel(llm='chat_llm', conversation_id="conversation_2")
# user_response_2 = user_model_2.run(user_prompt=user_prompt_2, 
#                                  task_prompt=task_prompt, 
#                                  buffer=buffer)

# print(user_response_2)

In [10]:
# # include user 1 feedback
# buffer.save_context(user={"content": "Your response is concise and includes key principles, but it could benefit from examples or more detailed explanations of each principle. Also, consider including possible challenges AI might face in maintaining balance between respecting human rights and considering cultural differences."}, user_message_id="conversation_1_user_message_1")
# # includee user 2 feedback
# buffer.save_context(user={"content": "Sure, your response is like a pizza with no pineapple. It's okay, but missing that extra spice! Try adding specific examples of how AI can respect human rights while considering local cultures. And don't forget to mention the limitations and challenges AI might face. 'Task Completed!'? Nah, 'Task Just Begun!'!"}, user_message_id="conversation_2_user_message_1")

In [11]:
# assistant_model_1 = AssistantModel(llm='chat_llm', 
#                                    conversation_id="conversation_1")
# assistant_response_1 = assistant_model_1.run(assistant_prompt=assistant_prompt, 
#                                              task_prompt=task_prompt,
#                                               buffer=buffer)
# print("assistant response 1")                                              
# print(assistant_response_1)

# print()

# assistant_model_2 = AssistantModel(llm='chat_llm', 
#                                    conversation_id="conversation_2")
# assistant_response_2 = assistant_model_2.run(assistant_prompt=assistant_prompt, 
#                                              task_prompt=task_prompt,
#                                              buffer=buffer)
# print("assistant response 2")   
# print(assistant_response_2)

In [12]:
# # revised assistant response
# buffer.save_context(assistant={"content": "AI should uphold universal human rights, while respecting local cultural norms and laws. For instance, supporting LGBTQ rights universally, while mindfully adapting responses to culturally-sensitive regions. Balancing these may pose challenges, requiring nuanced programming and regular updates."},
#                     assistant_message_id="conversation_1_assistant_message_2")

# buffer.save_context(assistant={"content": "AI, while respecting universal human rights, adapts communication to local cultures. For example, discussing gender equality in a respectful manner within a conservative society. However, striking the balance can be challenging."},
#                     assistant_message_id="conversation_2_assistant_message_2")

In [13]:
# # imagine we reached a time out now -> we want to run the meta prompt
# meta_model = MetaPromptModel(llm='chat_llm')
# meta_response = meta_model.run(meta_prompt=meta_prompt, 
#                                task_prompt=task_prompt,
#                                buffer=buffer)
# print(meta_response)

In [14]:
# # prompting a custom chat model
# from scai.custom_chat_models.crfm import crfmChatLLM

# CRFM_API_KEY = "p4z0j9adj6edJOWBMnEqfPBZxAXlfOGd"

# chat_llm = crfmChatLLM(model_name="openai/gpt-4-0314", 
#                    crfm_api_key=CRFM_API_KEY, 
#                    max_tokens=100, # need to be careful with this one
#                    num_completions=1,
#                    request_timeout=10,
#                    verbose=False,
#                    temperature=0.9,
#                    )